In [ ]:
import os

!pip install numpy==1.26.4 --quiet
os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 24.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
!pip install pmdarima --quiet

In [56]:
import pandas as pd
import numpy as np
import warnings
import joblib
import plotly.express as px
from google.colab import drive
from datetime import date, timedelta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.metrics import silhouette_score
from google.colab import files
from datetime import timedelta
from pmdarima import auto_arima
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 500

from google.colab import auth
import gspread
from google.auth import default
from google.cloud import bigquery
client = bigquery.Client(project='dev-sd-lake')

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [9]:
all_marketplaces_amazon_price_df = pd.read_csv('/content/drive/My Drive/PSI_Modeling_Data/Amazon_Keepa/keepa_price_amazon_20250614.csv')
all_marketplaces_amazon_price_df.head()

,date,retail_price,discounted_price,rating,asin,marketplace,category
0,2024-12-15,2.98,NaN,NaN,B003JKFEL8,US,shampoo
1,2024-12-16,3.08,NaN,NaN,B003JKFEL8,US,shampoo
2,2024-12-17,3.07,NaN,NaN,B003JKFEL8,US,shampoo
3,2024-12-18,3.00,NaN,NaN,B003JKFEL8,US,shampoo
4,2024-12-19,2.97,NaN,NaN,B003JKFEL8,US,shampoo


In [10]:
unique_marketplaces = all_marketplaces_amazon_price_df.marketplace.unique()
print(all_marketplaces_amazon_price_df.shape, '\n', unique_marketplaces)

(4363920, 7) 
 ['US' 'GB' 'JP' 'DE']


In [11]:
def check_date_continuity(df):
    issues = []

    grouped = df.groupby(['marketplace', 'category', 'asin'])

    for (marketplace, category, asin), group in grouped:
        group = group.sort_values('date')
        actual_dates = set(pd.to_datetime(group['date']).dt.date)

        start = pd.to_datetime(group['date'].min())
        end = pd.to_datetime(group['date'].max())
        expected_dates = set((start + timedelta(days=i)).date() for i in range((end - start).days + 1))

        missing_dates = sorted(expected_dates - actual_dates)

        if missing_dates:
            issues.append({
                'marketplace': marketplace,
                'category': category,
                'asin': asin,
                'missing_count': len(missing_dates),
                'missing_dates': missing_dates[:5]
            })

    if not issues:
        print("✅ All ASINs have continuous dates with no gaps.")
    else:
        print(f"⚠️ Found {len(issues)} ASINs with missing date gaps:\n")
        for issue in issues:
            print(f"🔍 {issue['marketplace']} | {issue['category']} | {issue['asin']}:")
            print(f"   Missing {issue['missing_count']} dates. Example: {issue['missing_dates']}")
            print("")


def compute_avg_timeseries_from_summary(all_data: pd.DataFrame, summary_df: pd.DataFrame) -> pd.DataFrame:
    """
    Computes average price time series per (marketplace, category) using pre-selected
    price series for each ASIN from summary_df.
    """

    merged = all_data.merge(summary_df, on=['marketplace', 'category', 'asin'], how='inner')

    # Assign chosen series as a new column
    merged['chosen_price'] = merged.apply(lambda row: row[row['chosen_series']], axis=1)


    # Pivot to have each ASIN as a column
    pivot_df = (
        merged.pivot_table(index=['marketplace', 'category', 'date'],
                           columns='asin',
                           values='chosen_price')
        .reset_index()
    )

    # Compute average across ASINs
    pivot_df['average_price'] = pivot_df.drop(columns=['marketplace', 'category', 'date']).mean(axis=1)

    # Final output
    avg_df = pivot_df[['marketplace', 'category', 'date', 'average_price']]
    return avg_df


def forecast_all_marketplace_category(avg_df, forecast_days=30):
    forecast_list = []

    # Set ARIMA search space (you can customize these)
    P_RANGE = range(0, 3)
    D_RANGE = [1]
    Q_RANGE = range(0, 3)
    SP_RANGE = range(0, 2)
    SD_RANGE = [0, 1]
    SQ_RANGE = range(0, 2)
    SEASONAL_PERIOD = 7  # Weekly seasonality

    grouped = avg_df.groupby(['marketplace', 'category'])

    for (marketplace, category), group in grouped:
        ts = group.sort_values('date')
        ts_indexed = ts.set_index('date')['average_price']

        try:
            model = auto_arima(
                ts_indexed,
                start_p=0, max_p=max(P_RANGE),
                start_q=0, max_q=max(Q_RANGE),
                d=None, max_d=max(D_RANGE),
                start_P=0, max_P=max(SP_RANGE),
                start_Q=0, max_Q=max(SQ_RANGE),
                D=None, max_D=max(SD_RANGE),
                seasonal=True,
                m=SEASONAL_PERIOD,
                stepwise=True,
                suppress_warnings=True,
                error_action="ignore",
                trace=False,
                n_jobs=-1,
            )

            future_dates = [ts_indexed.index[-1] + timedelta(days=i) for i in range(1, forecast_days + 1)]
            forecast = model.predict(n_periods=forecast_days)

            df_forecast = pd.DataFrame({
                'marketplace': marketplace,
                'category': category,
                'date': future_dates,
                'forecast_price': forecast
            })

            forecast_list.append(df_forecast)

        except Exception as e:
            print(f"❌ Model failed for {marketplace} | {category}: {e}")

    return pd.concat(forecast_list, ignore_index=True)



def compare_future_past_averages(average_df, forecast_df):
    """
    Compares the average price of the past 3 months with the forecasted 1-month average
    for each marketplace-category combination.

    Parameters:
    - average_df: DataFrame with columns ['marketplace', 'category', 'date', 'avg_price']
    - forecast_df: DataFrame with columns ['marketplace', 'category', 'date', 'forecast']

    Returns:
    - summary_df: DataFrame with past and future average prices per group
    """

    # Ensure date columns are datetime
    average_df['date'] = pd.to_datetime(average_df['date'])
    forecast_df['date'] = pd.to_datetime(forecast_df['date'])

    results = []

    # Group both datasets by marketplace-category
    grouped_past = average_df.groupby(['marketplace', 'category'])
    grouped_future = forecast_df.groupby(['marketplace', 'category'])

    for group_key in grouped_past.groups.keys():
        if group_key not in grouped_future.groups:
            continue  # Skip if no forecast for this group

        past_group = grouped_past.get_group(group_key)
        future_group = grouped_future.get_group(group_key)

        # Define the last 3 months from the latest date
        max_past_date = past_group['date'].max()
        past_3m_start = max_past_date - timedelta(days=89)

        past_3m_data = past_group[
            (past_group['date'] >= past_3m_start) &
            (past_group['date'] <= max_past_date)
        ]

        past_avg = past_3m_data['average_price'].mean()
        future_avg = future_group['forecast_price'].mean()

        results.append({
            'marketplace': group_key[0],
            'category': group_key[1],
            'past_3_month_avg': round(past_avg, 2),
            'future_1_month_avg': round(future_avg, 2)
        })

    return pd.DataFrame(results)

In [12]:
all_forecast_factors_df = pd.DataFrame()

for marketplace_code in unique_marketplaces:
    print(f"🚀 Processing {marketplace_code}...")

    amazon_price_df = all_marketplaces_amazon_price_df[all_marketplaces_amazon_price_df.marketplace == marketplace_code]

    check_date_continuity(amazon_price_df)
    amazon_price_df['date'] = pd.to_datetime(amazon_price_df['date'])

  # Forward fill first, then backward fill for 'retail_price' grouped by marketplace-category-asin
    amazon_price_df['retail_price'] = (
      amazon_price_df
      .groupby(['marketplace', 'category', 'asin'])['retail_price']
      .transform(lambda x: x.ffill().bfill()))

    print('Selecting series per asin ...')

    final_frames = []
    summary_records = []

    grouped = amazon_price_df.groupby(['marketplace', 'category', 'asin'])

    for (marketplace, category, asin), group in grouped:
        retail = group['retail_price']
        discount = group['discounted_price']

        # Determine selection based on null presence
        if retail.isna().sum() == 0:
            selected_series = retail
            selected_type = 'retail_price'
        elif discount.isna().sum() == 0:
            selected_series = discount
            selected_type = 'discounted_price'
        else:
            selected_series = retail
            selected_type = 'retail_price'  # default

        # Assign the selected series
        group = group.copy()
        group['final_price'] = selected_series.values

        final_frames.append(group)
        summary_records.append({
            'marketplace': marketplace,
            'category': category,
            'asin': asin,
            'chosen_series': selected_type,
            'retail_nulls': int(retail.isna().sum()),
            'discount_nulls': int(discount.isna().sum())
        })

    # Combine the final price dataframe
    final_data = pd.concat(final_frames, ignore_index=True)

    # Save summary decision table
    summary_df = pd.DataFrame(summary_records)
    summary_df = summary_df.drop(['retail_nulls', 'discount_nulls'], axis = 1)

    print('Calculating per sub category / hscode series...')

    avg_df = compute_avg_timeseries_from_summary(amazon_price_df, summary_df)
    print(avg_df.shape)

    print('Forecasting the sub category / hscode series...')

    forecast_df = forecast_all_marketplace_category(avg_df, forecast_days=30)
    print(forecast_df.shape)

    print('Calculating forecast factors using forecasts and past 3 month averages for each sub category ...')
    final_df = compare_future_past_averages(avg_df, forecast_df)
    final_df['forecast_to_avg_factor'] = final_df['future_1_month_avg'] / final_df['past_3_month_avg']
    final_df['forecast_to_avg_factor'] = np.where(final_df['forecast_to_avg_factor'] >= 2, np.random.uniform(1.75,2), final_df['forecast_to_avg_factor'])
    final_df = final_df.drop(['past_3_month_avg', 'future_1_month_avg'], axis = 1)
    print(final_df.shape)

    fig = px.histogram(final_df, x='forecast_to_avg_factor', title='Distribution - forecast_to_avg_factor')
    fig.show()

    print('Concatenating current marketplaces to net dataframe ...')
    all_forecast_factors_df = pd.concat([all_forecast_factors_df, final_df], ignore_index=True)

🚀 Processing US...
✅ All ASINs have continuous dates with no gaps.
Selecting series per asin ...
Calculating per sub category / hscode series...
(14220, 4)
Forecasting the sub category / hscode series...
(2370, 4)
Calculating forecast factors using forecasts and past 3 month averages for each sub category ...
(79, 3)


Concatenating current marketplaces to net dataframe ...
🚀 Processing GB...
✅ All ASINs have continuous dates with no gaps.
Selecting series per asin ...
Calculating per sub category / hscode series...
(25560, 4)
Forecasting the sub category / hscode series...
(4260, 4)
Calculating forecast factors using forecasts and past 3 month averages for each sub category ...
(142, 3)


Concatenating current marketplaces to net dataframe ...
🚀 Processing JP...
✅ All ASINs have continuous dates with no gaps.
Selecting series per asin ...
Calculating per sub category / hscode series...
(25560, 4)
Forecasting the sub category / hscode series...
(4260, 4)
Calculating forecast factors using forecasts and past 3 month averages for each sub category ...
(142, 3)


Concatenating current marketplaces to net dataframe ...
🚀 Processing DE...
✅ All ASINs have continuous dates with no gaps.
Selecting series per asin ...
Calculating per sub category / hscode series...
(25560, 4)
Forecasting the sub category / hscode series...
(4260, 4)
Calculating forecast factors using forecasts and past 3 month averages for each sub category ...
(142, 3)


Concatenating current marketplaces to net dataframe ...


In [63]:
all_forecast_factors_df.sample(5)

,marketplace,category,forecast_to_avg_factor
256,GB,"toys, games, crafts - other",1.016024
60,US,premixed alcohol drinks,0.670442
245,GB,sports equipment,0.920780
531,DE,electronics - other,1.002098
66,US,women's bottoms,1.139052


In [52]:
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1PPIC2TTRGap722HfZvFl9azMnJCoiNsORb8GDC1OyAQ/edit?pli=1&gid=1089158624#gid=1089158624"
sh = gc.open_by_url(spreadsheet_url)
worksheet = sh.worksheet("Product_HSCode_Mapped_Full.csv")
data = worksheet.get_all_records()
cat_sub_cat_combos_df = pd.DataFrame(data)
cat_sub_cat_combos_df = cat_sub_cat_combos_df[['Product Category', 'Product Sub Category']]

total_domains = list(all_marketplaces_amazon_price_df.marketplace.unique())
domain_df = pd.DataFrame({
    "domain": total_domains
})
# Add a dummy key to both for cross join
domain_df["key"] = 1
cat_sub_cat_combos_df["key"] = 1

# Perform cross join
cross_joined_df = pd.merge(domain_df, cat_sub_cat_combos_df, on="key").drop("key", axis=1)
cross_joined_df = cross_joined_df.drop('Product Category', axis = 1)
cross_joined_df = cross_joined_df.rename(columns = {'domain' : 'marketplace', 'Product Sub Category' : 'category'})
cross_joined_df['category'] = cross_joined_df['category'].str.lower()
cross_joined_df = pd.merge(cross_joined_df, all_forecast_factors_df, 'left', on = ['marketplace', 'category'])

In [54]:
print(cross_joined_df.shape)

(588, 3)


In [55]:
!pip install -q sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 27.9 MB/s eta 0:00:00


In [57]:
# Step 3: Define the imputation function
def impute_forecast_factors(df, category_col='category', marketplace_col='marketplace', value_col='forecast_to_avg_factor'):
    """
    Imputes missing forecast_to_avg_factor using category semantic similarity and marketplace constraints.

    Parameters:
    df (pd.DataFrame): DataFrame with 'marketplace', 'category', and 'forecast_to_avg_factor'

    Returns:
    pd.DataFrame: A new DataFrame with missing values imputed
    """

    df = df.copy()

    # Load sentence transformer model
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # Encode all unique categories
    unique_categories = df[category_col].unique()
    category_embeddings = dict(zip(
        unique_categories,
        model.encode(unique_categories, show_progress_bar=True)
    ))

    # Progress bar
    tqdm.pandas()

    # Function to impute a single row
    def impute_row(row):
        if not np.isnan(row[value_col]):
            return row[value_col]  # Already filled

        current_cat = row[category_col]
        current_mkt = row[marketplace_col]

        # Filter same marketplace and non-null values
        marketplace_df = df[(df[marketplace_col] == current_mkt) & (df[value_col].notnull())]
        if marketplace_df.empty:
            return np.nan  # No data to infer from

        # Compute similarity with other categories
        sims = []
        for _, r in marketplace_df.iterrows():
            sim = cosine_similarity(
                [category_embeddings[current_cat]],
                [category_embeddings[r[category_col]]]
            )[0][0]
            sims.append((r[category_col], r[value_col], sim))

        # Sort by similarity and get top 3
        top3 = sorted(sims, key=lambda x: x[2], reverse=True)[:3]
        valid_values = [val for _, val, _ in top3 if not pd.isna(val)]

        if not valid_values:
            return np.nan
        else:
            return np.mean(valid_values)

    # Apply the imputation row-wise
    df[value_col] = df.progress_apply(impute_row, axis=1)
    return df

In [58]:
all_forecast_factors_df = impute_forecast_factors(cross_joined_df)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/588 [00:00<?, ?it/s]

In [69]:
all_forecast_factors_df.to_csv('/content/drive/MyDrive/PSI_Modeling_Data/Amazon_Keepa/all_forecast_factors_df.csv', index=False)